<a href="https://colab.research.google.com/github/YDHYDHDONG/geography_visualization/blob/master/%EA%B5%90%ED%86%B5_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A1%9C_%EC%A7%80%EB%A6%AC_%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **교통 데이터 : 지리 데이터 시각화**

## **데이터 살펴보기**

In [1]:
import pandas as pd
import plotly.express as px

#경고 무시
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pd.read_csv('/content/drive/MyDrive/데이터분석/geography_visualization/data/서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv의 사본',encoding='cp949')

In [3]:
data.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202307,1호선,동대문,710,31,11581,2095,9027,6829,14177,...,8930,511,2412,0,216,0,0,0,0,20230803
1,202307,1호선,동묘앞,221,2,3012,1090,3682,5042,7336,...,4195,104,1649,0,272,0,0,0,0,20230803
2,202307,1호선,서울역,577,37,8434,9396,13088,51243,40143,...,13987,2297,2598,1,219,0,0,0,0,20230803
3,202307,1호선,시청,49,5,2381,4825,3784,22736,7558,...,4285,287,1085,0,12,0,0,0,0,20230803
4,202307,1호선,신설동,392,35,8967,2356,9520,8509,21822,...,9505,317,2244,0,0,0,0,0,0,20230803


## **질문**
- 승차 인원이 가장 많은 역은?
- 연도별로 혹은 월별로 승차 인원 추이에 차이가 있는가?
- 시간대별로 가장 승차인원이 많은 역은?
- 지하철역 시간대별 인원 유형 군집화
- 지도에 분석 결과를 시각화하기

## **데이터 전처리 파트**

- 날짜 컬럼 추가

In [4]:
data['연도'] = pd.to_datetime(data['사용월'], format='%Y%m').dt.year
data['월'] = pd.to_datetime(data['사용월'],format='%Y%m').dt.month
data.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자,연도,월
0,202307,1호선,동대문,710,31,11581,2095,9027,6829,14177,...,2412,0,216,0,0,0,0,20230803,2023,7
1,202307,1호선,동묘앞,221,2,3012,1090,3682,5042,7336,...,1649,0,272,0,0,0,0,20230803,2023,7
2,202307,1호선,서울역,577,37,8434,9396,13088,51243,40143,...,2598,1,219,0,0,0,0,20230803,2023,7
3,202307,1호선,시청,49,5,2381,4825,3784,22736,7558,...,1085,0,12,0,0,0,0,20230803,2023,7
4,202307,1호선,신설동,392,35,8967,2356,9520,8509,21822,...,2244,0,0,0,0,0,0,20230803,2023,7


- 2018년 이후, 2호선만 필터링

In [5]:
data = data.query('호선명 == "2호선" and 연도 >=2018')

- 지하철역명 통일

In [6]:
sorted(data['지하철역'].unique())

['강남',
 '강변(동서울터미널)',
 '건대입구',
 '교대(법원.검찰청)',
 '구로디지털단지',
 '구의(광진구청)',
 '낙성대',
 '낙성대(강감찬)',
 '당산',
 '대림(구로구청)',
 '도림천',
 '동대문역사문화공원',
 '동대문역사문화공원(DDP)',
 '뚝섬',
 '문래',
 '방배',
 '봉천',
 '사당',
 '삼성(무역센터)',
 '상왕십리',
 '서울대입구(관악구청)',
 '서초',
 '선릉',
 '성수',
 '시청',
 '신답',
 '신당',
 '신대방',
 '신도림',
 '신림',
 '신설동',
 '신정네거리',
 '신촌',
 '아현',
 '양천구청',
 '역삼',
 '영등포구청',
 '왕십리(성동구청)',
 '용답',
 '용두(동대문구청)',
 '을지로3가',
 '을지로4가',
 '을지로입구',
 '이대',
 '잠실(송파구청)',
 '잠실나루',
 '잠실새내',
 '종합운동장',
 '충정로(경기대입구)',
 '한양대',
 '합정',
 '홍대입구']

In [7]:
data['지하철역'] = [i[0] for i in data['지하철역'].str.split('(')]
sorted(data['지하철역'].unique())

['강남',
 '강변',
 '건대입구',
 '교대',
 '구로디지털단지',
 '구의',
 '낙성대',
 '당산',
 '대림',
 '도림천',
 '동대문역사문화공원',
 '뚝섬',
 '문래',
 '방배',
 '봉천',
 '사당',
 '삼성',
 '상왕십리',
 '서울대입구',
 '서초',
 '선릉',
 '성수',
 '시청',
 '신답',
 '신당',
 '신대방',
 '신도림',
 '신림',
 '신설동',
 '신정네거리',
 '신촌',
 '아현',
 '양천구청',
 '역삼',
 '영등포구청',
 '왕십리',
 '용답',
 '용두',
 '을지로3가',
 '을지로4가',
 '을지로입구',
 '이대',
 '잠실',
 '잠실나루',
 '잠실새내',
 '종합운동장',
 '충정로',
 '한양대',
 '합정',
 '홍대입구']

- 승차 인원만 추출하기

In [8]:
on_col = [i for i in data.columns if '승차' in i]
data = data[['사용월','연도','월','지하철역']+on_col]

data.head()

,사용월,연도,월,지하철역,04시-05시 승차인원,05시-06시 승차인원,06시-07시 승차인원,07시-08시 승차인원,08시-09시 승차인원,09시-10시 승차인원,...,18시-19시 승차인원,19시-20시 승차인원,20시-21시 승차인원,21시-22시 승차인원,22시-23시 승차인원,23시-24시 승차인원,00시-01시 승차인원,01시-02시 승차인원,02시-03시 승차인원,03시-04시 승차인원
10,202307,2023,7,강남,120,9359,18503,40112,64256,52713,...,327480,215011,172551,196756,177694,80992,13773,3,0,0
11,202307,2023,7,강변,27,8936,28877,78164,117728,82763,...,61291,43557,32792,27539,19267,7874,1058,1,0,0
12,202307,2023,7,건대입구,290,16965,22074,53127,91248,61438,...,85337,64419,65838,72636,74202,45555,7559,3,0,0
13,202307,2023,7,교대,15,2693,11535,25250,36820,27461,...,133358,62654,51239,53658,42819,15754,3171,2,0,0
14,202307,2023,7,구로디지털단지,369,38505,53882,144838,184498,112257,...,210532,101581,65184,55202,41572,18845,3463,6,0,0


## **분석**

### [1] 승하차 인원이 가장 많은 역은?

- 합계 컬럼 만들기

In [9]:
data['합계'] = data[on_col].sum(axis=1)

data.head()

,사용월,연도,월,지하철역,04시-05시 승차인원,05시-06시 승차인원,06시-07시 승차인원,07시-08시 승차인원,08시-09시 승차인원,09시-10시 승차인원,...,19시-20시 승차인원,20시-21시 승차인원,21시-22시 승차인원,22시-23시 승차인원,23시-24시 승차인원,00시-01시 승차인원,01시-02시 승차인원,02시-03시 승차인원,03시-04시 승차인원,합계
10,202307,2023,7,강남,120,9359,18503,40112,64256,52713,...,215011,172551,196756,177694,80992,13773,3,0,0,2408376
11,202307,2023,7,강변,27,8936,28877,78164,117728,82763,...,43557,32792,27539,19267,7874,1058,1,0,0,933338
12,202307,2023,7,건대입구,290,16965,22074,53127,91248,61438,...,64419,65838,72636,74202,45555,7559,3,0,0,1060476
13,202307,2023,7,교대,15,2693,11535,25250,36820,27461,...,62654,51239,53658,42819,15754,3171,2,0,0,852909
14,202307,2023,7,구로디지털단지,369,38505,53882,144838,184498,112257,...,101581,65184,55202,41572,18845,3463,6,0,0,1629379


- 지하철 월평균 승차 인원 구하기

In [10]:
data_mean = data.groupby('지하철역')[['합계']].mean().reset_index().rename({'합계':'월평균'}, axis=1).sort_values('월평균', ascending=False)
data_mean

,지하철역,월평균
0,강남,2.475218e+06
42,잠실,2.185502e+06
49,홍대입구,1.895644e+06
27,신림,1.815902e+06
4,구로디지털단지,1.656332e+06
16,삼성,1.496685e+06
26,신도림,1.481450e+06
20,선릉,1.463946e+06
18,서울대입구,1.390678e+06
33,역삼,1.286969e+06


In [11]:
fig = px.bar(data_frame = data_mean, x = '지하철역', y = '월평균', title = '지하처역별 월평균 승차인원')
fig.show()

### [2] 연도별로 혹은 원별로 승차 인원 추이에 차이가 있는가?

In [12]:
data.head()

,사용월,연도,월,지하철역,04시-05시 승차인원,05시-06시 승차인원,06시-07시 승차인원,07시-08시 승차인원,08시-09시 승차인원,09시-10시 승차인원,...,19시-20시 승차인원,20시-21시 승차인원,21시-22시 승차인원,22시-23시 승차인원,23시-24시 승차인원,00시-01시 승차인원,01시-02시 승차인원,02시-03시 승차인원,03시-04시 승차인원,합계
10,202307,2023,7,강남,120,9359,18503,40112,64256,52713,...,215011,172551,196756,177694,80992,13773,3,0,0,2408376
11,202307,2023,7,강변,27,8936,28877,78164,117728,82763,...,43557,32792,27539,19267,7874,1058,1,0,0,933338
12,202307,2023,7,건대입구,290,16965,22074,53127,91248,61438,...,64419,65838,72636,74202,45555,7559,3,0,0,1060476
13,202307,2023,7,교대,15,2693,11535,25250,36820,27461,...,62654,51239,53658,42819,15754,3171,2,0,0,852909
14,202307,2023,7,구로디지털단지,369,38505,53882,144838,184498,112257,...,101581,65184,55202,41572,18845,3463,6,0,0,1629379


In [12]:
data.query('연도 <= 2022')